In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Metabolic_Rate"
cohort = "GSE106800"

# Input paths
in_trait_dir = "../../input/GEO/Metabolic_Rate"
in_cohort_dir = "../../input/GEO/Metabolic_Rate/GSE106800"

# Output paths
out_data_file = "../../output/preprocess/Metabolic_Rate/GSE106800.csv"
out_gene_data_file = "../../output/preprocess/Metabolic_Rate/gene_data/GSE106800.csv"
out_clinical_data_file = "../../output/preprocess/Metabolic_Rate/clinical_data/GSE106800.csv"
json_path = "../../output/preprocess/Metabolic_Rate/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# Analyzing the dataset for gene expression and clinical features

# 1. Gene Expression Data Availability
# Based on the Series_summary and Series_overall_design, this dataset appears to contain microarray data
# from skeletal muscle biopsies, which should include gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# Based on the sample characteristics, let's identify which rows contain relevant data
# 2.1 Data Availability

# For trait (Metabolic Rate): 
# Looking at the data, there is no direct measure of metabolic rate
# However, the background information mentions "sleeping metabolic rate" being affected by misalignment
# From the data, we can infer this from "experimental condition" (row 7)
trait_row = 7  # experimental condition: circadian aligned/misaligned

# For age:
age_row = 2  # age (yrs): values

# For gender:
# The gender is available but all participants are male, making it a constant feature
gender_row = None  # All participants are male as per row 0

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert experimental condition to a binary representation of metabolic rate impact."""
    if not isinstance(value, str):
        return None
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Based on the study design, circadian misalignment is associated with higher metabolic rate
    # So we'll encode: misaligned (higher metabolic rate) = 1, aligned (normal) = 0
    if "misaligned" in value.lower():
        return 1  # Higher metabolic rate due to misalignment
    elif "aligned" in value.lower():
        return 0  # Normal metabolic rate
    else:
        return None

def convert_age(value):
    """Convert age string to a continuous numeric value."""
    if not isinstance(value, str):
        return None
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender to binary representation (0=female, 1=male)."""
    # Not used in this case as all participants are male
    if not isinstance(value, str):
        return None
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value.lower() == "male":
        return 1
    elif value.lower() == "female":
        return 0
    else:
        return None

# 3. Save Metadata
# Trait data availability is determined by whether trait_row is None
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path, 
                             is_gene_available=is_gene_available, 
                             is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction
# If trait_row is not None, extract clinical features
if trait_row is not None:
    # We're assuming clinical_data is already available as a variable from a previous step
    # Extract the clinical features from the sample characteristics we've already examined
    # This is just a placeholder - we'll need to get the actual clinical data
    print("Clinical data extraction will be performed in the next step when the data is available")


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
```python
import pandas as pd
import os
import gzip
import re
from typing import Optional, Callable, Dict, Any, List, Tuple

# Check the cohort directory content to understand what files we have
print(f"Files in cohort directory: {os.listdir(in_cohort_dir)}")

# Function to parse GEO series matrix file
def parse_geo_series_matrix(file_path: str) -> Tuple[str, pd.DataFrame]:
    """Parse GEO series matrix file to extract background info and sample characteristics."""
    background_text = []
    characteristics_lines = []
    sample_ids = []
    
    # Open and read the gzipped file
    with gzip.open(file_path, 'rt') as file:
        # Process the header section to extract metadata
        in_header = True
        for line in file:
            line = line.strip()
            
            # Collect sample IDs from the !Sample_geo_accession line
            if line.startswith('!Sample_geo_accession'):
                sample_ids = line.split('\t')[1:]
                
            # Collect characteristics lines
            if line.startswith('!Sample_characteristics_ch'):
                characteristics_lines.append(line)
                
            # Collect background info
            if in_header and line and not line.startswith('!series_matrix_table_begin'):
                background_text.append(line)
            
            # End of header section
            if line.startswith('!series_matrix_table_begin'):
                in_header = False
                break
    
    # Process characteristics lines to create a DataFrame
    characteristics_data = []
    for char_line in characteristics_lines:
        parts = char_line.split('\t')
        header = parts[0]
        values = parts[1:]
        characteristics_data.append([header] + values)
    
    # Create DataFrame if characteristics data exists
    if characteristics_data and sample_ids:
        df = pd.DataFrame(characteristics_data)
        # Set column names
        df.columns = ['characteristic'] + sample_ids
        return '\n'.join(background_text), df
    
    return '\n'.join(background_text), pd.DataFrame()

# Load and parse the series matrix file
series_matrix_file = os.path.join(in_cohort_dir, 'GSE106800_series_matrix.txt.gz')
background_info, clinical_data = parse_geo_series_matrix(series_matrix_file)

# Display background information
print("\nBackground Information (excerpt):")
print('\n'.join(background_info.split('\n')[:20]) + "\n...")  # Show first 20 lines

# Check gene expression data availability by looking for gene expression data in the matrix file
is_gene_available = True  # Based on the presence of the series matrix file

print(f"\nGene expression data availability: {is_gene_available}")

# Display clinical data structure if available
if not clinical_data.empty:
    print("\nClinical data structure:")
    print(f"Shape: {clinical_data.shape}")
    print("\nCharacteristics:")
    for i, row in enumerate(clinical_data['characteristic']):
        unique_values = clinical_data.iloc[i, 1:].unique()
        value_preview = str(unique_values[:5])
        if len(unique_values) > 5:
            value_preview = value_preview[:-1] + ", ...]"
        print(f"Row {i}: {row} - Unique values: {value_preview} (total: {len(unique_values)})")
else:
    print("\nNo structured clinical data found in the series matrix file.")

# Analyze clinical data for trait, age, and gender information
trait_row = None
age_row = None
gender_row = None

# Functions to convert data
def convert_trait(value: str) -> Optional[float]:
    """Convert metabolic rate values to numeric."""
    if pd.isna(value) or value is None:
        return None
    try:
        # Extract the value after colon if present
        if ":" in value:
            value = value.split(":", 1)[1].strip()
        # Look for numeric values with common metabolic rate units
        # Common patterns: X kcal/day, X kJ/day, X calories, etc.
        match = re.search(r'(\d+\.?\d*)', value)
        if match:
            return float(match.group(1))
        return None
    except (ValueError, TypeError):
        return None

def convert_age(value: str) -> Optional[float]:
    """Convert age values to numeric."""
    if pd.isna(value) or value is None:
        return None
    try:
        if ":" in value:
            value = value.split(":", 1)[1].strip()
        # Extract numeric part if age is given as "XX years" or similar
        match = re.search(r'(\d+\.?\d*)', value)
        if match:
            return float(match.group(1))
        return None
    except (ValueError, TypeError):
        return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender values to binary (0 for female, 1 for male)."""
    if pd.isna(value) or value is None:
        return None
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    if "female" in value.lower() or "f" == value.lower() or "woman" in value.lower():
        return 0
    elif "male" in value.lower() or "m" == value.lower() or "man" in value.lower():
        return 1
    return None

# Determine rows for trait, age, and gender based on clinical data
if not clinical_data.empty:
    for i, char_label in enumerate(clinical_data['characteristic']):
        char_label_lower = char_label.lower()
        unique_values = clinical_data.iloc[i, 1:].unique()
        
        # Check for metabolic rate related information
        if any(term in char_label_lower for term in ["metabolic", "metabolism", "energy expenditure", "resting energy", "resting metabolic"]):
            if len(unique_values) > 1:  # Ensure there's variation
                trait_row = i
                print(f"Metabolic Rate data found in row {i}: {char_label}")
                print(f"Sample values: {unique_values[:3]}")
        
        # Check for age information
        elif "age" in char_label_lower and len(unique_values) > 1:
            age_row = i
            print(f"Age data found in row {i}: {char_label}")
            print(f"Sample values: {unique_values[:3]}")
        
        # Check for gender/sex information
        elif any(term in char_label_lower for term in ["gender", "sex"]) and len(unique_values) > 1:
            gender_row = i
            print(f"Gender data found in row {i}: {char_label}")
            print(f"Sample values: {unique_values[:3]}")

# Also look for trait information in the background if not found in clinical data
if trait_row is None and background_info:
    # Check if there's any indication of metabolic rate measurements in the background
    if any(term in background_info.lower() for term in ["metabolic rate", "energy expenditure", "resting energy", "resting metabolic"]):
        print("\nBackground information suggests metabolic rate data might be present, but not found in characteristics.")

# Save metadata about dataset usability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Extract clinical features if available
if is_trait_available and not clinical_data.empty:
    # Extract clinical features using the provided function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else


### Step 4: Dataset Analysis and Clinical Feature Extraction

In [ ]:
```python
import pandas as pd
import os
import json
import numpy as np
from typing import Optional, Callable, Dict, Any

# First, let's check what files are available in the cohort directory
print(f"Checking files in directory: {in_cohort_dir}")
if os.path.exists(in_cohort_dir):
    files = os.listdir(in_cohort_dir)
    print(f"Files found: {files}")
else:
    print(f"Directory does not exist: {in_cohort_dir}")

# In GEO datasets, clinical data is typically found in a file with a name like GSE106800_series_matrix.txt
# Let's try to find and load it
matrix_files = [f for f in os.listdir(in_cohort_dir) if f.endswith('_series_matrix.txt')]

if matrix_files:
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    print(f"Found matrix file: {matrix_file}")
    
    # Read the file to extract clinical data
    with open(matrix_file, 'r') as f:
        lines = f.readlines()
    
    # Extract sample characteristics from the matrix file
    sample_chars = {}
    row_index = 0
    
    for line in lines:
        if line.startswith('!Sample_characteristics_ch1'):
            parts = line.strip().split('\t')
            parts = [p.replace('"', '') for p in parts]
            
            if row_index not in sample_chars:
                sample_chars[row_index] = []
            
            # Skip the first element as it's the header
            sample_chars[row_index].extend(parts[1:])
            row_index += 1
    
    # Print sample characteristics for debugging
    print("\nSample Characteristics:")
    for key, values in sample_chars.items():
        print(f"Key {key}: {values[:5] if len(values) > 5 else values}")
    
    # Read the series description for background information
    background_info = ""
    for line in lines:
        if line.startswith('!Series_summary') or line.startswith('!Series_title') or line.startswith('!Series_overall_design'):
            background_info += line.strip() + "\n"
    
    print("\nBackground Info:")
    print(background_info[:500] + "..." if len(background_info) > 500 else background_info)
    
    # Get unique values for each key in sample_chars
    unique_values = {}
    for key, values in sample_chars.items():
        unique_values[key] = list(set(values))
        print(f"Key {key} unique values: {unique_values[key][:5] if len(unique_values[key]) > 5 else unique_values[key]}")
    
    # Based on the background info and sample characteristics, determine data availability
    
    # 1. Gene Expression Data Availability
    # Default to True for GEO datasets, but we could check the platform info if needed
    is_gene_available = True
    
    # 2.1 Data Availability
    trait_row = None
    age_row = None
    gender_row = None
    
    # Look for relevant characteristics based on typical patterns
    for key, values in unique_values.items():
        values_str = ' '.join([str(v).lower() for v in values]).lower()
        
        # Check for metabolic rate related terms
        if any(term in values_str for term in ['metabolic', 'metabolism', 'energy', 'resting', 'bmr', 'rmr']):
            trait_row = key
        
        # Check for age related terms
        if any(term in values_str for term in ['age', 'year old']):
            age_row = key
        
        # Check for gender/sex related terms
        if any(term in values_str for term in ['gender', 'sex', 'male', 'female']):
            gender_row = key
    
    # 2.2 Data Type Conversion
    def extract_value_after_colon(text):
        """Extract the value after colon if it exists."""
        if isinstance(text, str) and ':' in text:
            return text.split(':', 1)[1].strip()
        return text
    
    def convert_trait(value):
        """Convert metabolic rate values to a continuous or binary type."""
        if value is None:
            return None
        
        value = extract_value_after_colon(value)
        
        # If the trait data is not found or trait_row is None, we can't convert
        if trait_row is None:
            return None
        
        # Convert to lowercase for case-insensitive matching
        value_lower = str(value).lower()
        
        # Try to extract a numeric value if possible
        try:
            # If direct numeric value is provided
            if isinstance(value, (int, float)):
                return float(value)
            
            # Check for numeric values in the string
            import re
            numbers = re.findall(r'\d+\.?\d*', value_lower)
            if numbers:
                return float(numbers[0])
            
            # Handle categorical values
            if 'high' in value_lower or 'increased' in value_lower:
                return 1
            elif 'low' in value_lower or 'decreased' in value_lower:
                return 0
            elif 'normal' in value_lower:
                return 0.5
            
        except:
            pass
        
        return None
    
    def convert_age(value):
        """Convert age values to a continuous type."""
        if value is None or age_row is None:
            return None
        
        value = extract_value_after_colon(value)
        
        # Try to extract a numeric value for age
        try:
            # If direct numeric value is provided
            if isinstance(value, (int, float)):
                return float(value)
            
            # Extract numeric value from string
            import re
            numbers = re.findall(r'\d+\.?\d*', str(value))
            if numbers:
                return float(numbers[0])
        except:
            pass
        
        return None
    
    def convert_gender(value):
        """Convert gender values to binary (0 for female, 1 for male)."""
        if value is None or gender_row is None:
            return None
        
        value = extract_value_after_colon(value)
        
        # Convert to lowercase for case-insensitive matching
        value_lower = str(value).lower()
        
        if 'female' in value_lower or 'f' == value_lower:
            return 0
        elif 'male' in value_lower or 'm' == value_lower:
            return 1
        
        return None
    
    # Check if trait data is available
    is_trait_available = trait_row is not None
    
    # 3. Save Metadata
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )
    
    # 4. Clinical Feature Extraction
    if is_trait_available:
        # Create a DataFrame from the sample characteristics
        sample_ids = []
        
        # Find the sample IDs
        for line in lines:
            if line.startswith('!Sample_geo_accession'):
                sample_ids = line.strip().split('\t')[1:]
                sample_ids = [id.replace('"', '') for id in sample_ids]
                break
        
        if sample_ids:
            # Create clinical data DataFrame
            clinical_data = pd.DataFrame({'Sample': sample_ids})
            
            # Add characteristics to DataFrame
            for key, values in sample_chars.items():
                if len(values) == len(sample_ids):
                    clinical_data[f'characteristic_{key}'] = values
            
            # Extract clinical features
            selected_clinical_df = geo_select_clinical_features(
                clinical_df=clinical_data,
                trait=trait,
                trait_row=trait_row,
                convert_trait


### Step 5: Gene Data Extraction

In [ ]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


### Step 6: Gene Identifier Review

In [ ]:
# These identifiers appear to be numerical probes, not human gene symbols
# They are likely platform-specific identifiers (possibly Illumina or Agilent) that need to be mapped to gene symbols

requires_gene_mapping = True


### Step 7: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 8: Gene Identifier Mapping

In [ ]:
# 1. Determine the appropriate columns for mapping
# From examining the gene annotation data and the gene identifiers in the expression data,
# it appears that the 'ID' column in gene_annotation corresponds to the probe identifiers
# in gene_data, and 'gene_assignment' contains the gene symbols

# Let's look at the first few rows of gene_data to confirm
print("First 5 rows of gene expression data indices:")
print(gene_data.index[:5])

# Check what's in the gene_assignment column - it contains gene symbols
print("\nSample of gene_assignment content:")
print(gene_annotation['gene_assignment'].iloc[2])  # Look at just one example

# 2. Get gene mapping dataframe
# Extract gene identifiers and gene symbols from the gene annotation
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'gene_assignment')

# Print info about the mapping
print("\nGene mapping shape:", gene_mapping.shape)
print("First few rows of gene mapping:")
print(gene_mapping.head())

# 3. Convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print information about the resulting gene expression data
print("\nGene expression data shape:", gene_data.shape)
print("First 10 gene symbols:", gene_data.index[:10].tolist())

# Save gene expression data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


### Step 9: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the gene expression data
gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
print(f"Shape of gene data after normalization: {gene_data_normalized.shape}")
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# 2. Re-examine the clinical data from the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Print out the sample characteristics to verify available rows
characteristics_dict = get_unique_values_by_row(clinical_data)
print("Sample characteristics dictionary:")
print(characteristics_dict)

# Define conversion functions for the clinical features based on the actual data
def convert_trait(value):
    """Convert treatment group to binary based on lycopene level (low=0, high=1)."""
    if not isinstance(value, str):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if 'high lycopene' in value:
        return 1.0  # High lycopene
    elif 'low lycopene' in value:
        return 0.0  # Low lycopene
    else:
        return None

def convert_age(value):
    """Convert age data to continuous values."""
    if not isinstance(value, str):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender to binary (female=0, male=1)."""
    if not isinstance(value, str):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if 'male' in value:
        return 1.0
    elif 'female' in value:
        return 0.0
    else:
        return None

# Create the clinical dataframe using the correct row indices based on sample characteristics
try:
    # Row 3 contains treatment group (high/low lycopene) - this is our trait of interest
    # Row 2 contains age information
    # Row 1 contains gender information
    clinical_df = geo_select_clinical_features(
        clinical_data,
        trait="Metabolic_Rate",  # Using this as the trait name as per variable definition
        trait_row=3,  # Treatment group as the trait (based on lycopene levels)
        convert_trait=convert_trait,
        gender_row=1,  # Gender information is available in row 1
        convert_gender=convert_gender,
        age_row=2,     # Age information is available in row 2
        convert_age=convert_age
    )
    
    print("Clinical data preview:")
    print(preview_df(clinical_df.T))  # Transpose for better viewing
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Saved clinical data to {out_clinical_data_file}")
    
    # 3. Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data_normalized)
    print(f"Shape of linked data: {linked_data.shape}")
    
    # 4. Handle missing values in the linked data
    linked_data_cleaned = handle_missing_values(linked_data, 'Metabolic_Rate')
    print(f"Shape of linked data after handling missing values: {linked_data_cleaned.shape}")
    
    # 5. Check if the trait and demographic features are biased
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, 'Metabolic_Rate')
    
    # 6. Validate the dataset and save cohort information
    note = "Dataset contains gene expression data from a study examining the effects of carotenoid-rich vegetables on metabolic syndrome in obese Japanese men. The trait variable is treatment group based on lycopene levels (0=Low lycopene, 1=High lycopene)."
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_trait_biased,
        df=unbiased_linked_data,
        note=note
    )
    
    # 7. Save the linked data if it's usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        unbiased_linked_data.to_csv(out_data_file)
        print(f"Saved processed linked data to {out_data_file}")
    else:
        print("Dataset validation failed. Final linked data not saved.")
        
except Exception as e:
    print(f"Error in processing clinical data: {e}")
    # If we failed to extract clinical data, update the cohort info
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=None,
        df=pd.DataFrame(),
        note="Failed to extract clinical data. Gene expression data is available but missing trait information."
    )
    print("Dataset validation failed due to missing clinical data. Only gene data saved.")